In [ ]:
# 形態素分析ライブラリーMeCab と 辞書(mecab-ipadic-NEologd)のインストール 
!apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl python-mecab > /dev/null
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null 
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n > /dev/null 2>&1
!pip install mecab-python3 > /dev/null
 
# シンボリックリンクによるエラー回避
!ln -s /etc/mecabrc /usr/local/etc/mecabrc

Cloning into 'mecab-ipadic-neologd'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 75 (delta 5), reused 54 (delta 0), pack-reused 0
Unpacking objects: 100% (75/75), done.


In [ ]:
import os
import MeCab
import re
import subprocess
import sys
path = "-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd -b 819200"
m = MeCab.Tagger(path)
noun_list = ''
pos_list = ''
from google.colab import drive
drive.mount('/content/drive')

def ommit(linestr):
  line = linestr.rstrip('\n')
  t_lists = line.split('\t')
  term = t_lists[0] + ' '
  if t_lists[0] == 'EOS' or t_lists[0] == '' or t_lists[0] == 'と' or t_lists[0] == 'の':
    term = ''
    return term
  for hinsi in ['記号', '助詞,連体化', '助詞,副詞化', '助詞,係助詞', '助詞,接続助詞', '助詞,並立助詞', '助詞,副助詞／並立助詞／終助詞', '助詞,格助詞,引用', '助詞,格助詞,一般,*,*,*,へ,ヘ,エ', '助詞,格助詞,一般,*,*,*,に,ニ,ニ', '助詞,格助詞,一般,*,*,*,を,ヲ,ヲ', '助詞,格助詞,一般,*,*,*,で,デ,デ', '助詞,格助詞,一般,*,*,*,が,ガ,ガ', '助詞,格助詞,一般,*,*,*,から,カラ,カラ', '助詞,副助詞,*,*,*,*,まで,マデ,マデ', '連体詞', '接頭詞,名詞接続', '特殊', '形容詞,自立,*,*,形容詞・アウオ段,基本形,ない,ナイ,ナイ', '接頭詞,数接続', '名詞,数', '名詞,接尾', '名詞,非自立,副詞可能', '名詞,非自立,助動詞語幹', '名詞,接尾,助数詞', '名詞,代名詞', '助動詞', '名詞,非自立,一般', '動詞,非自立', '動詞,自立,*,*,サ変・スル', '動詞,接尾', '動詞,自立,*,*,一段,基本形,できる,デキル,デキル', '動詞,自立,*,*,五段・ラ行,基本形,係る,カカル,カカル', '助詞,格助詞,連語,*,*,*,に' ]:
    if t_lists[1].find(hinsi) > -1:
      term = ''
      return term
  for stopword in ['ヶ所', 'ヵ所', 'カ所', '箇所', 'ヶ月', 'ヵ月', 'カ月', '箇月', '十', '百', '千', '万', '億', '兆', 'もの', '明治', '大正', '昭和', '平成', '令和']:
    if t_lists[0].find(stopword) > -1:
      term = ''
      return term
  return linestr

for bunrui in range(1,51):
  lssentence = " ".join(['ls','/content/drive/My\ Drive/Colab\ Notebooks/houreibunrui20210510/%02d/*.nml' % bunrui])
  output = subprocess.getoutput(lssentence)
  tolists = output.split('\n')
  for tolist in tolists:
    print(tolist)
    text_data = open(tolist, "rb").read()
    decoded_text = text_data.decode('utf-8')
    text = re.sub('\n\n', '\n', decoded_text)
    with open(tolist[:125] + '.poslist', 'a') as f:
      parses = m.parse(text)
      tolistformecab = re.sub(" ", "\ ", tolist)
      mecabsentence = " ".join(['cat', tolistformecab, '|', 'mecab', path, '>', 'temp.poslist'])
      subprocess.getoutput(mecabsentence)
      with open('temp.poslist', encoding='utf8') as ftemp:
        for linestr in ftemp:
          f.write(ommit(linestr))
      !rm temp.poslist
f.close()

In [ ]:
import os
import sys
import re
import subprocess
from google.colab import drive
drive.mount('/content/drive')
f=open('/content/drive/My Drive/Colab Notebooks/houreibunrui20210510/bunruiwall-sort-uniq-poslist2.csv')
lines=f.readlines()
f.close()
bfs = []
sums= []
nums = []
mods = []
bunrui = 0
bunruii = 1
for line in lines:
  bunrui = int(line[0:2])
  fn = line[3:64]
  size = int(line[65:].replace("\n", ""))
  bfs.append((bunrui, fn, size))
#bfsは(分類、分かち書き後のファイルネーム、サイズ)のリスト
sums = [0, 0]
nums = [0]
mods = [0]
bunruilens = [0]
bunruilen = 0
for bfsi in bfs:
  if bfsi[0]  == bunruii:
#bfsiは(分類、分かち書き後のファイルネーム、サイズ)のタプル
    sums[bunruii] = sums[bunruii] + bfsi[2]
    bunruilen += 1
  elif sums[bunruii]//1690000  == 0:
    nums.append(1)
    mods.append(0)
    sums.append(bfsi[2])
    bunruilens.append(bunruilen)
    print(bunruii, 1, 0, bunruilen)
    bunruilen = 1
    bunruii = bfsi[0]
  else:
    nums.append(sums[bunruii]//1690000 + 1)
    mods.append(((sums[bunruii] % -1690000)//(nums[bunruii] - 1)) * -1)
    sums.append(bfsi[2])
    bunruilens.append(bunruilen)
    print(bunruii, sums[bunruii], sums[bunruii]//1690000 + 1, ((sums[bunruii] % -1690000)//(nums[bunruii] - 1)) * -1, bunruilen)
    bunruilen = 1
    bunruii = bfsi[0]
if sums[bunruii]//1690000  == 0:
  nums.append(1)
  mods.append(0)
  bunruilens.append(bunruilen)
  print(bunruii, 1, 0)
else:
  nums.append(sums[bunruii]//1690000 + 1)
  mods.append(((sums[bunruii] % -1690000)//(nums[bunruii] - 1)) * -1)
  bunruilens.append(bunruilen)
  print(bunruii, sums[bunruii]//1690000 + 1, ((sums[bunruii] % -1690000)//(nums[bunruii] - 1)) * -1, bunruilen)
print(len(sums), len(nums), len(mods), len(bunruilens))
print(bunruilens)


In [ ]:
wakati = ['']
wakatilist = []
wakatilistbw = []
numw = 0
sumfw = 0
sumbw = 0
t = 0
def writeallwakati(bunruii, numw, wakatilist):
  from pprint import pprint
  import pickle
  import json
  texts = []
  text = []
  fn = '%02d-%02dw.allposlist.pkl' % (bunruii, numw)
  for tolist in wakatilist:
    if not tolist == '':
      with open('/content/drive/My Drive/Colab Notebooks/houreibunrui20210510/%02d/' % bunruii + tolist[0], "rb") as fr:
        text = [row.decode('utf-8').split(",")[0].replace('\t', ',') for row in fr]
        texts += set(text)
  results = {}
  for textline in sorted(set(texts)):
    word, pos = textline.split(",")
    results.setdefault(word, []).append(pos)
  open(fn, 'wb').write(pickle.dumps(results))

for bunruii in range(1, 51):
  start = sum(bunruilens[0:bunruii])
  end = sum(bunruilens[0:bunruii + 1]) - 1
  print(bunruii, start, end)
  for t1, bfsi in enumerate(bfs[start:end + 1]):
    sumfw = sumfw + int(bfsi[2])
    if sumfw > 1590000:
      print('reversed', bunruii, sumfw)
      wakati.append((bunruii, numw, wakatilist))
      writeallwakati(bunruii, numw, wakatilist)
      wakatilist = []
      wakatilistbw = []
      numw += 1
      sumfw = 0
      for t2, bfsbw in enumerate(reversed(bfs[start:start + t1])):
        sumbw = sumbw + int(bfsbw[2])
        print(sumbw, int(mods[bunruii])/(int(nums[bunruii]) - 1))
        if sumbw > int(mods[bunruii])/(int(nums[bunruii]) - 1):
          sumfw = sumbw - int(bfsbw[2]) + int(bfsi[2]) 
          print('sumfw is %d' % sumfw)
          wakatilist = wakatilistbw[:]
          wakatilistbw = []
          sumbw = 0
          break
        wakatilistbw.append((bfsbw[1][:53] + ".poslist", bfsbw[2]))
        print(t2, sumbw)
      else:
        sumfw = sumbw + int(bfsi[2]) 
        print('sumbw is %d' % sumbw)
        wakatilist = wakatilistbw[:]
        wakatilistbw = []
        sumbw = 0
    wakatilist.append((bfsi[1][:53] + ".poslist", bfsi[2]))
    print(t1, sumfw)
  wakati.append((bunruii, numw, wakatilist))
  writeallwakati(bunruii, numw, wakatilist)
  wakatilist = []
  wakatilistbw = []
  sumfw = 0
  numw = 0
  sumbw = 0


In [ ]:
from __future__ import print_function
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, concatenate
from keras.layers import LSTM, GRU, SimpleRNN
from keras.layers.core import Dropout, Reshape, Permute, RepeatVector, Flatten, Lambda
from keras.layers.normalization import BatchNormalization as BN
from keras.layers.embeddings import Embedding
from keras.optimizers import RMSprop, Adam, Adagrad, Nadam, SGD, Adadelta, Adamax
from keras.utils.data_utils import get_file
from keras.callbacks import ReduceLROnPlateau
from keras import backend as K
import numpy as np
import random
import sys
import glob
from time import gmtime, strftime
import copy
import pickle
import json
import tensorflow as tf
import math
import os
import re
import subprocess
from google.colab import drive
drive.mount('/content/drive')
def pred(wakatialllists, fnb, fnp, bunrui):
  rng = 5
  writetexts = []
  poslistdic = {}
  print(fnb)
  with open('/content/drive/My Drive/Colab Notebooks/houreibunrui20210510/' + bunrui + '/' + fnp, "rb") as fr:
    text = [row.decode('utf-8').split(",")[0].replace('\t', ',') for row in fr]
  for textline in sorted(set(text)):
    word, pos = textline.split(",")
    poslistdic.setdefault(word, []).append(pos)


  for wakatialllis in wakatialllists:
    wakatialllist = wakatialllis.strip()
    print("start to loading term_vec and poslist")
    ansindx = []
    term_vec = []
    term_vec_lists = pickle.loads(open('/content/' + wakatialllist + '.term_vec.pkl', 'rb').read())
    allposlistdic = pickle.loads(open('/content/' + wakatialllist + '.allposlist.pkl', 'rb').read())
    text = open('/content/drive/My Drive/Colab Notebooks/houreibunrui20210510/' + wakatialllist[0:2] + '/' + fnb, 'r').read().replace('\n', ' ').split()
    if len(text) < 11:
      break
    picking_up = []
    for term_vec_list in term_vec_lists:
      term_vec.append([term_vec_list[0], term_vec_list[2]])
      ansindx.append(term_vec_list[0])
    print(len(term_vec))
    term_vec_lists = []
    for i in range(rng, len(text) - rng, 1):
      try:
        head = list(map(lambda x:term_vec[ansindx.index(x)][1], text[i-rng:i] )) 
        tail = list(map(lambda x:term_vec[ansindx.index(x)][1], text[i+1:i+rng] )) 
      except ValueError as e:
        continue
      head.extend(tail)
      x = np.array(head)
      y = text[i]
      picking_up.append( (x, y, text[i-rng:i+rng]) )
    print(len(picking_up))
    answers   = []
    texts     = []
    writetext = []
    sentences = []
    for dbi, picked in enumerate(picking_up):
      x, y,  pure_text = picked
      sentences.append(x)
      answers.append(y)
      texts.append(pure_text)
    if len(sentences) == 0:
      break
    X = np.zeros((len(sentences), len(sentences[0]), 128), dtype=np.float64)
    for i, sentence in enumerate(sentences):
      if i%10000 == 0:
        print("building training vector... iter %d"%i)
      for t, vec in enumerate(sentence):
        X[i, t, :] = vec


    model_type = './models' + wakatialllist + '/snapshot.000000030.model'
    print("model type is %s"%model_type)
    model  = load_model(model_type)
    results = model.predict(X)
    itext = 0
    j = 0
    for sent, xtext, result in zip(sentences, texts, results):
      itext = itext + 1
      while not text[itext] == xtext[5]:
        writetext.append(text[itext] + ':,')
        itext = itext + 1
      wtext = xtext[5] + ':'
      for i,f in sorted([(i,f) for i,f in enumerate(result.tolist())], key=lambda x:x[1]*-1):
        if f < 0.99:
          writetext.append(wtext)
          break
        if xtext[5] != ansindx[i] and set(poslistdic[xtext[5]]) & set(allposlistdic[ansindx[i]]):
          wtext = wtext + ansindx[i] + ';' + str(f) + ','
        j += 1
        if j % 5 == 0:
          writetext.append(wtext)
          break
    writetexts.append(writetext)
  return writetexts
dictwakatiall = {}
%cd /content/drive/My\ Drive/Colab\ Notebooks/houreibunrui20210510/01-50wakatiall/
lswakatiall = " ".join(['ls','*.wwakatiall'])
output = subprocess.getoutput(lswakatiall)
%cd /content/
for fni in output.split('\n'):
  if fni[0:2] in dictwakatiall:
    dictwakatiall[fni[0:2]].append(fni.strip())
  else:
    dictwakatiall.update({fni[0:2] : [fni.strip()]})
f=open('/content/drive/My Drive/Colab Notebooks/houreibunrui20210510/bunruiwall-sort-uniq2.csv')
lines=f.readlines()
f.close()
writetext = []
writetexts = []
outputs = []
for line in lines:
  wakatialllists = dictwakatiall[line[0:2]]
  fnb = line[3:64]
  fnp = line[3:56] + '.poslist'
  bunrui = line[0:2]
  writetexts = pred(wakatialllists, fnb, fnp, bunrui)
  with open('pred' + fnb + '.txt', 'a') as fwt:
    for ls in zip(*writetexts):
      fwt.write('|'.join(ls) + "\n")
!zip /content/drive/My\ Drive/Colab\ Notebooks/houreibunrui20210510/wwakati.zip *.txt